In [7]:
from graph import GraphDB
import random

wa_latitude_range = (-35.0, -13.5) 
wa_longitude_range = (112.5, 129.0)  

def generate_random_location():
    latitude = random.uniform(wa_latitude_range[0], wa_latitude_range[1])
    longitude = random.uniform(wa_longitude_range[0], wa_longitude_range[1])
    return [latitude, longitude]

embedding_dictionary = {}
for i in range(1, 21):
    ref = f"ref{i}"
    embedding_dictionary[ref] = {
        "embedding": [random.uniform(0, 1) for _ in range(3)],
        "location": generate_random_location()
}
    
graph_db = GraphDB(embedding_dictionary)

In [15]:
import folium
from folium import plugins
from folium.features import DivIcon
from ipywidgets import interact, widgets
import networkx as nx  


def get_node_positions(reference, num_nodes, max_distance):
    query_result = graph_db.closest(reference, num_nodes, max_distance)
    
    G = graph_db.draw_graph(query_result)
    return nx.get_node_attributes(G, "location"), nx.get_edge_attributes(G,"similarity")

def create_map(map_center):
    m = folium.Map(location=map_center, zoom_start=5)
    return m

def query_and_display(reference, num_nodes, max_distance):
    
    edge_text_group = folium.FeatureGroup()
    node_positions,node_similarity = get_node_positions(reference, num_nodes, max_distance)
    map_center = node_positions[reference]    
    
    # create map with centered on reference 
    m = create_map(map_center)
    
    # add markers for ref positions on map
    for ref,position in node_positions.items():
        
        # plot ref marker
        folium.CircleMarker(location=position,popup=reference,fill_color='blue').add_to(m)
        
        # plot ref label
        folium.Marker(location=position,popup=reference,icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt">{ref}</div>',
        )).add_to(m)
        
        if ref != reference:
            edge = folium.PolyLine([node_positions[reference],position],popup=str(node_similarity[(reference, ref)]))
            edge.add_to(m)
            edge_text = plugins.PolyLineTextPath(edge,text = str(node_similarity[(reference, ref)]),center=True,attributes={'fill': '007DEF', 'font-weight': 'bold', 'font-size': '24'})
            edge_text_group.add_child(edge_text)
            
      
    m.add_child(edge_text_group)

    display(m)

reference_input = widgets.Text(value='ref1', description='Enter Reference:')
number_slider = widgets.IntSlider(value=2, min=1, max=10, step=1, description='Closest Tenders:')
distance_slider = widgets.FloatSlider(value=100, min=1, max=300, step=1, description='Max Distance:')

interact(query_and_display, reference=reference_input, num_nodes=number_slider, max_distance=distance_slider)


interactive(children=(Text(value='ref1', description='Enter Reference:'), IntSlider(value=2, description='Clos…

<function __main__.query_and_display(reference, num_nodes, max_distance)>